In [3]:
from unsloth import FastLanguageModel

max_seq_length = 1248 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model_type = "phi" # llama, phi-3, gemma

model = FastLanguageModel.from_pretrained(
    model_name = "saracandu/SmolGRPO-135M", # MODEL OR ADAPTER FOLDER
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla V100-PCIE-32GB. Num GPUs = 1. Max memory: 31.739 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


OSError: Can't load tokenizer for 'saracandu/SmolGRPO-135M'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'saracandu/SmolGRPO-135M' is the correct path to a directory containing all relevant files for a LlamaTokenizerFast tokenizer.

In [ ]:
from datasets import load_dataset

eval_dataset = load_dataset('saracandu/eureka-rebus-grpo', data_files = ['id_test.csv'], split="train")

In [ ]:
# Single-example generation

stop_token_id = model.config.eos_token_id
if model_type == "gemma":
    stop_token = "<|eot_id|>"
    stop_token_id = tokenizer.encode(stop_token)[0]

In [ ]:
from tqdm import tqdm

if model_type == "llama":
    tokenizer.padding_side = "right"
elif model_type in ("phi-3", "gemma"):
    tokenizer.padding_side = "left"

In [ ]:
import re

regex_word_guess = '- \[.* = (.*)'
regex_firstpass = 'Prima lettura: (.*)'
regex_solution_word = "\d+ = (.*)"
regex_solution = "Soluzione: (.*)"

def parse_generation(data):
    try:
        word_guesses = ";".join(re.findall(regex_word_guess, data))
    except:
        word_guesses = ""
    try:
        first_pass = re.findall(regex_firstpass, data)[0]
    except:
        first_pass = ""
    try:
        solution_words = ";".join(re.findall(regex_solution_word, data))
    except:
        solution_words = ""
    try:
        solution = re.findall(regex_solution, data)[0]
    except:
        solution = ""
    
    return {
        "word_guesses": word_guesses,
        "first_pass": first_pass,
        "solution_words": solution_words,
        "solution": solution,
    }


In [ ]:
# eval_dataset['conversations'][5][0]['value']

In [ ]:
# parse_generation(eval_dataset['conversations'][5][1]['value'])

In [ ]:
# inputs = tokenizer(eval_dataset['conversations'][5][0]['value'], return_tensors="pt")["input_ids"].to('cuda:0')
# outputs = model.generate(input_ids = inputs, max_new_tokens = 500, use_cache = True, eos_token_id = stop_token_id)
# model_generations = tokenizer.batch_decode(outputs)
# print(model_generations[0])

In [ ]:
# from unsloth import FastLanguageModel

# max_seq_length = 1248 # Choose any! We auto support RoPE Scaling internally!
# dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
# load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# model_type = "gemma" # llama, phi-3, gemma

# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "gsarti/gemma-2-2b-rebus-solver-fp16", # MODEL OR ADAPTER FOLDER
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     load_in_4bit = load_in_4bit,
# )
# FastLanguageModel.for_inference(model)

# from datasets import load_dataset

# eval_dataset = load_dataset('gsarti/eureka-rebus', 'llm_sft', data_files=["id_test.jsonl", "ood_test.jsonl"], split = "train")

# # Single-example generation

# stop_token_id = model.config.eos_token_id
# if model_type == "gemma":
#     stop_token = "<|eot_id|>"
#     stop_token_id = tokenizer.encode(stop_token)[0]

# ex_idx = 5
# example = eval_dataset[ex_idx]["conversations"][0]
# inputs = tokenizer.apply_chat_template(
#     [
#         {"role": "user", "content": example["value"]}
#     ],
#     add_generation_prompt=True,
#     return_tensors = "pt",
#     padding=True,
#     truncation=True,
# )
# inputs = inputs.to('cuda:0')
# outputs = model.generate(input_ids = inputs, max_new_tokens = 500, use_cache = True, eos_token_id = stop_token_id)
# model_generations = tokenizer.batch_decode(outputs)
# print(model_generations[0])



# import re

# regex_word_guess = '- \[.* = (.*)'
# regex_firstpass = 'Prima lettura: (.*)'
# regex_solution_word = "\d+ = (.*)"
# regex_solution = "Soluzione: (.*)"

# def parse_generation(ex_idx, ex):
#     try:
#         word_guesses = ";".join(re.findall(regex_word_guess, ex))
#     except:
#         word_guesses = ""
#     try:
#         first_pass = re.findall(regex_firstpass, ex)[0]
#     except:
#         first_pass = ""
#     try:
#         solution_words = ";".join(re.findall(regex_solution_word, ex))
#     except:
#         solution_words = ""
#     try:
#         solution = re.findall(regex_solution, ex)[0]
#     except:
#         solution = ""
#     return {
#         "idx": ex_idx,
#         "word_guesses": word_guesses,
#         "first_pass": first_pass,
#         "solution_words": solution_words,
#         "solution": solution,
#     }




# import pandas as pd
# from tqdm import tqdm

# # Create gold parsed outputs

# results = []
# for ex_idx, ex in tqdm(enumerate(eval_dataset), total=len(eval_dataset)):
#     gold_output = ex["conversations"][1]["value"]
#     parsed_output = parse_generation(ex_idx, gold_output)
#     results.append(parsed_output)

In [ ]:
# from tqdm import tqdm

# if model_type == "llama":
#     tokenizer.padding_side = "right"
# elif model_type in ("phi-3", "gemma"):
#     tokenizer.padding_side = "left"

# results = []
# batch_size = 134
# for i in tqdm(range(0, len(eval_dataset), batch_size), total=len(eval_dataset)//batch_size):
#     batch = eval_dataset[i:i+batch_size]
#     print(batch['conversations'][0])
#     if model_type == "llama":
#         input = [[{"role": "user", "content": example[0]["value"]}] for example in batch["conversations"]]
    # elif model_type == "phi-3":
        # print(batch['conversation'])
        # input = [[item[0]] for item in batch["conversations"]]

    # inputs = tokenizer.apply_chat_template(
    #     input,
    #     tokenize = True,
    #     add_generation_prompt = True, # Must add for generation
    #     return_tensors = "pt",
    #     padding=True,
    #     truncation=True,
    #     return_dict=True
    # )
    # inputs = {k: v.to("cuda") for k, v in inputs.items()}
    # outputs = model.generate(**inputs, max_new_tokens = 500, use_cache = True, eos_token_id = stop_token_id)

    # model_generations = tokenizer.batch_decode(outputs)
    # for ex_idx, ex in enumerate(model_generations):
    #     out_dic = parse_generation(ex_idx + i, ex)
    #     if i == 0 and ex_idx <= 5:
    #         print(ex)
    #         print(out_dic)
    #     results.append(out_dic)

# qui

In [ ]:
# Single-example generation

stop_token_id = model.config.eos_token_id
if model_type == "gemma":
    stop_token = "<|eot_id|>"
    stop_token_id = tokenizer.encode(stop_token)[0]

ex_idx = 5
example = eval_dataset[ex_idx]
inputs = tokenizer(
    example['prompt'],
    return_tensors = "pt",
    padding=True,
    truncation=True,
)
inputs = inputs.to('cuda:0')
outputs = model.generate(input_ids = inputs["input_ids"], max_new_tokens = 500, use_cache = True, eos_token_id = stop_token_id)
model_generations = tokenizer.batch_decode(outputs)
parse_generation(model_generations[0])

In [ ]:
model_generations

In [ ]:
eval_dataset

In [ ]:
# Seleziona un subset di dimensione 10
subset = eval_dataset.select(range(10))

# Verifica il subset
print(subset)

In [ ]:
results = []

for ex_idx in tqdm(range(len(subset)), total=len(subset)):
    # Ottieni il primo esempio di conversazione (senza batch)
    examples = subset[ex_idx]
    prompts = examples['conversations'][0]

    inputs = tokenizer.apply_chat_template(
    [
        {"role": "user", "content": prompts["value"]}
    ],
    add_generation_prompt=True,
    return_tensors = "pt",
    padding=True,
    truncation=True,
    )
    inputs = inputs.to('cuda:0')
    
    # print(example)  # Puoi usare questo per controllare il contenuto dell'esempio

    # input = [[item[0]] for item in examples]
    
    # # A questo punto puoi fare inferenza su `input`, ad esempio:
    # inputs = tokenizer.apply_chat_template(
    #     input,
    #     tokenize=True,
    #     add_generation_prompt=True,  # Aggiungi il prompt per la generazione
    #     return_tensors="pt",
    #     padding=True,
    #     truncation=True,
    #     return_dict=True
    # )

    # # Inferenza sul modello
    outputs = model.generate(inputs, max_new_tokens=500, use_cache=True, eos_token_id=stop_token_id)
    model_generations = tokenizer.batch_decode(outputs)
    
    # # Aggiungi il risultato alla lista dei risultati
    results.append(parse_generation(model_generations[0]))
    
    # # Opzionalmente, puoi fare qualcosa con i risultati, come stamparli:
    print(model_generations[0])


In [ ]:
results